### Gathering data

1. I loaded the given .csv file into a dataframe df_archive.
2. I downloaded the .tsv from the internet using Requests library and loaded into a dataframe df_image.
3. I directly loaded the json file into a dataframe df_json since I don't have a Twitter account and don't want to create one.

### Assessing data

#### Visual assessment

1. df_archive

When I looked at the dataframe, I found two startling issues:
a. The rating value is present in two separate columns: one for the numerator and another for the denominator. This is a structural issue. One variable in one column rule is not followed here.
b. Again, the dog stage is spread across four columns with the values as the column headers. This is another structural issue. Dog stage variable is represented in four columns instead of one.

2. df_image

I noticed that for some rows all the 3 predictions are not a breed of dog. For example, last row has 'orange', 'bagel', 'banana'. Since I am interested only in the rating of a dog image, I need only valid records where at least one of the predictions identify the image as a breed of dog. There is validity issue in this dataframe that needs to be fixed.

3. df_json

I noticed that some columns are redundant with df_archive and there are many columns here that I don't require for further analysis. The columns that I require can be combined with df_archive. Another tidiness issue.


So, with visual assessment, I identified the following 3 tidiness issue and 1 quality issue:

**Quality:**

There are some images in df_image that are not classified as that of a dog. (validity)

**Tidiness:**

One variable (i.e. rating) in two columns (rating_numerator and rating_denominator) in df_archive.

One variable (i.e. dog stage) in four columns (doggo, floofer, pupper, puppo) in df_archive.

retweet_count and favorite_count columns should be part of df_archive.

#### Programmatic assessment

1. df_archive

Using .info() I noticed that the dataframe contains some retweets. Since I am interested only in original tweets, all these retweets are invalid for my further analysis. Also, the datatype of timestamp is incorrect.

Using .value_counts() for the name column, I noticed that some names are not real name of dogs (example, 'a'). I further analyzed with visual assessment in excel file. There are few other names that are not dog names. The commonality with all these inaccurate name is they all start with small alphabets.

Using .sort_values() for rating numerator and denominator, I noticed that the values are all over the place. The denominator is not consistently 10. Some are in 100s. So does the numerator. I further analyzed with visual assessment in excel file. I noticed that some ratings are represented not on a scale of 10, but something like 143/130, 121/110 etc. There is consistency issue here. Sure, these inconsistent ratings can be standardized to a rating 10 scale. 

On further analysis of the rating data, I noticed that some ratings are inaccurately represented or rather extracted from the tweets. For example, if the rating in the tweet has a decimal value like 9.75/10, the extractor program assumed numerator to be 75 and denominator 10, which is inaccurate. There are few like these.

Also, I noticed that one of the denominator values is 0. On looking at the corresponding tweet, I realized that the extractor program must have been confused. Here is the tweet:
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho.
The rating extractor assumed the rating to be 960/0 instead of 13/10. 

So, there are few issues here with respect to the rating:
There is consistency issue where the rating is not standardized on a scale of 10.
There is accuracy issue where the correct rating is not extracted from the tweet. I want to further split the accuracy issue into two:
a. denominator 0 should be fixed first in order to avoid division by zero error when I am fixing the structural issue with the rating columns (i.e. representing rating in a single column by combining numerator and denominator).
b. all other rating accuracy issues can be fixed last. There are two reasons for this: First, identifying all the rating inaccuracies is a time consuming activity. Second, the invalid records (like retweets) will be eliminated. Identifying the inaccurate ratings only in the valid records at the end is much easier.

2. df_image

No new issue identified. However, after eliminating the retweets from df_archive and the non-dog images from df_image, both dataframes should be combined to have only the original tweets with dog image.

3. df_json

No new issue identified.

Following issues are identified as the end of assessment:

**Quality:**

Division by zero error: 960/0 rating instead of 13/10. (accuracy)

There are some retweets in df_archive. (validity)

Only original tweets with dog image should be considered for further analysis. There are some records that don't fall into this category. (validity)

Rating is not standardized. In other words, rating_denominator is not the same (10) for all records in df_archive. (consistency)

Name column in df_archive contains words other than a dog's name or 'None'. (accuracy)

Erroneous datatype for timestamp in df_achive.

Some incorrect ratings created by the program that extracted the ratings from the tweets. (accuracy)


### Cleaning data

I first took a copy of each dataframes:
df_archive_clean,
df_image_clean,
df_json_clean

Before tackling the tidiness issue, I want to solve the 1st quality issue: Division by zero for the reason mentioned before.

**Issue # 1: Division by zero error: 960/0 rating instead of 13/10**

Visually, I checked to make sure that there is only one record with numerator rating of 960 and denominator rating of 0. Then, I used replace function to change the numerator and denominator values to 13 and 10 respectively. Finally, I checked that the 960 and 0 rating values don't exist.

Next, I tackled the 3 tidiness issues one after another.

**Issue # 2: One variable (i.e. rating) in two columns (rating_numerator and rating_denominator) in df_archive.**

Rating is basically numerator/denominator. If the denominator is constant (i.e. 10), the rating can be represented just by the numerator value. Here, I am trying to solve the structural issue first. I created a new column 'rating' by dividing numerator value by denominator value. This will take care of some consistency issue as well (i.e. for example, both 11/10 and 121/110 will be represented as 1.1 in the new column). With the new rating column representing the rating in the decimal format, I dropped both the numerator and the denominator columns and used .info() to verify.

**Issue # 3: One variable (i.e. dog stage) in four columns (doggo, floofer, pupper, puppo) in df_archive.**

Four columns (doggo, floofer, pupper, puppo) have to be collapsed into a single column. I visually verified whether there is any record with more than one dog stage; unfortunately, there are few. First, I replaced the 'None' with empty string and then collapsed all the four columns into a single new column: dog_stage. Finally fixed the records with multiple dog stages. Used .info() and .value_counts() to verify.

**Issue # 4: retweet_count and favorite_count columns should be part of df_archive.**

If the two required columns retweet_count and favorite_count are part of df_archive, then the df_json can be discarded. I wanted to join both dataframes using the tweet_id as the key. Since, df_json has different column name for the key, I renamed it first, then selected a subset of columns from df_json and then used inner join to merge both dataframes. (Note: I also included 'created_at' column from df_json in order to later resolve the datatype issue with the 'timestamp' column in df_archive). I verified that  the new dataframe df_new now has the new columns using .head() and .info().

With all the identified tidiness issues resolved, next I started to clean the remaining quality issues.

**Issue # 5: There are some retweets in df_archive.**

From df_archive (new name df_new) , I selected only the records that have Null retweet_status_id. In other words, all the records that are a retweet are dropped. With the retweeted records gone, the three columns that are representing retweets are of no use, and so I dropped those 3 columns from the dataframe. I verified the record count and the columns using .info().

**Issue # 6: There are some images in df_image that are not classified as that of a dog.**

The logic I used to identify a dog image is at least one of the 3 predictions should have identified the image as that of a dog breed. In other words, at least one of p1_dog, p2_dog or p3_dog should be True. I selected only those records that satisfy this condition. Visually, I identified how many are all False for p1_dog, p2_dog and p3_dog from Excel and used it to cross verify the numbers using .info().

**Issue # 7: Only original tweets with dog image should be considered for further analysis. There are some records that don't fall into this category.**

With retweets eliminated from df_archive (new name df_new), it has only original tweets. df_image now has only dog images. I used inner join to merge both dataframes  into a new dataframe df to have only original tweets that have only dog images. After merging, I used .info() to verify all the columns are present in the new dataframe df.

**Issue # 8: Rating is not standardized. In other words, rating_denominator is not the same (10) for all records in df_archive.**

While solving Issue # 2 above, a new rating column was created. The value in this column is an absolute value. In order to standardize the rating to a scale of 10, the absolute value should be multiplied with 10. I used .value_counts to verify the new rating values.

**Issue # 9: Name column in df_archive contains words other than a dog's name or 'None'.**

As identified in assessment, the 'name' column has words starting with small alphabets that don't represent a dog's name. I identified all these words using regular expressions and replaced them with 'None'. 

**Issue # 10: Erroneous datatype for timestamp in df_achive.**

The dataframe df now has two columns 'timestamp' and 'created_at' that represents the same thing. Since 'timestamp' has an incorrect datatype, I dropped it and kept only 'created_at'. I used .info() to verify.

**Issue # 11: Some incorrect ratings created by the program that extracted the ratings from the tweets**

I used .value_counts() on the rating column to identify what values are there. While majority of the ratings are between 2 and 14, there are some odd values like 6.36, 8.18, 26, 27, 34.28, and 75. I created the rating column in Excel sheet by dividing numerator by denomination and multiplying by 10 and then identified these odd values and their corresponding tweets. From the tweets, I got the correct rating to replace the incorrect rating provided by the extractor program as follows:
- 7/11 instead of 10/10
- 9/11 instead of 14/10
- 26/10 instead of 11.26/10
- 27/10 instead of 11.27/10
- 75/10 instead of 9.75/10
- 24/7 for no rating provided

I used replace function to correct the ratings and dropped the record with no rating. Then, I verified with .value_counts(). The rating ranges from 0 to 14.

### Storing data

Finally, I stored the cleaned data into a .csv file using the pandas .to_csv() function.